# Laboratorio - Redes Neuronales

### David Soto / 17551

En este laboratorio se realizó un algoritmo de Redes Neuronales con el fin de poder hacer que algoritmo cree un moodelo que haya aprendido previamente a través de un set de datos de entrenamiento para poder predecir a que grupo pertenece un sub set de datos.

En este caso se nos brindó de un set de datos con la información de imágenes de ropa en forma de arreglos de valores de pixeles que conformaban cada imagen. Junto a los datos se proporcionó que un valor que a través de un diccionario determinaba el tipo de prenda o accesorio que representaba cada sub conjunto de datos en forma de imagen.

#### Cargamos los modulos que nos van a servir para entrenar a nuestra Red Neuronal

In [ ]:
import numpy as np
import pandas as pd
import pickle
from functools import reduce
from scipy import optimize as op
import matplotlib.pyplot as plt

#### Se carga el modulo de python con los metodos para realizar el algoritmo de construcción y gradiente de la Red Neuronal

In [ ]:
import RedesNeuronales as rn

#### Lectura, mezcla y división de datos para el Training Set, Test Set & Cross Validation Set

In [ ]:
## Cargamos datos
# Iniciamos a probar con datos de training y de test de fashion
# Se cargan los datos
datos1 = pd.read_csv('fashion-mnist_train.csv')
datos2 = pd.read_csv('fashion-mnist_test.csv')

# Unimos los 2 dataframes
frames = [datos1, datos2]
datos = pd.concat(frames)

# Revolvemos las filas del dataframe
datos = datos.sample(frac = 1)

# Se determina la cantidad de datos totales
cantidad_datos = len(datos)

# Obtenemos partes de cada Data Set
datosTraining = datos.iloc[:int(cantidad_datos * 0.6), :]
datosTest = datos.iloc[int(cantidad_datos * 0.6):int(cantidad_datos * 0.8), :]
datosCrossValidation = datos.iloc[int(cantidad_datos * 0.8):, :]

#### Ahora se procede a separar los datos de caracteristicas del cada sub set  X y el valor esperado Y del Training Set

In [ ]:
# Se procesa el dataset separando X de Y de Training
X = datosTraining.iloc[:, 1:] / 1000.0 #Normalizacion de los datos
m, n = X.shape
y = np.asarray(datosTraining.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)

#### Se crea la arquitectura de la Red Neuronal que se usará 
##### En este caso se hicieron 2 modelos
- ##### 1ro. 1 Capa de entrada de 784 neuronas, 1 Capa oculta de 130 neuronas y 1 Capa de salida de 10 neuronas
- ##### 2do. 1 Capa de entrada de 784 neuronas, 1 Capa oculta de 110 neuronas y 1 Capa de salida de 10 neuronas

In [ ]:
# Se hace un set de la arquitectura de la red neuronal
NETWORK_ARCHITECTURE = np.array([
    n,
    130,
    # 110,
    10
])

### OJO: Se recomienda que si se desea probar el código omita correr la parte de "Creación del modelo de Red Neuronal y Training" y saltarse hasta el título de "Test del Modelo de Red Neuronal creado y entrenado", debido a que el proceso de crear y entrenar al modelo tardan horas

## Creación del modelo de Red Neuronal y Training

#### Se lee un set de pesos Thetas Iniciales para las transiciones, que fue creado a partir de valores random, pero se guardo la configuración para poder replicar el experimento. Además obtenemos las shapes que se necesitan segun el modelo para poder aplanar las thetas

In [ ]:
# Se lee las thetas iniciale
# with (open("initial_model_110_Neurons", "rb")) as openfile:
with (open("initial_model_130_Neurons", "rb")) as openfile:
    while True:
        try:
            initialThetas = pickle.load(openfile)
        except EOFError:
            break

openfile.close()

flat_thetas = initialThetas

# Se extraen las shapes de las matrices pesos Thetas
theta_shapes = np.hstack((
    NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),
    (NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)
))

#### En este punto se procede a contruir el modelo de Red Neuronal y a entrenar el modelo, optimizando los valores iniciales de Thetas en cada iteración para obtener mejores valores de thetas que ayuden a generar un modelo que tenga un buen rendimiento

In [ ]:
# Se inicia con el entrenamiento del modelo de Red Neuronal
print("Optimazing...")
result = op.minimize(
    fun=rn.cost_function,
    x0=flat_thetas,
    args=(theta_shapes, X, Y),
    method='L-BFGS-B',
    jac=rn.cost_bayesian_neural_network,
    options={'disp': True, 'maxiter': 3000}
)
print("Optimized")

#### Se escribe el resultado del modelo optimizado de peso de Thetas en un archivo

In [ ]:
# Se escribe el resultado en un archivo
#outfile = open("model_trained_110_Neurons", "wb")
outfile = open("model_trained_130_Neurons", "wb")
pickle.dump(result.x, outfile)
outfile.close()

## Test del Modelo de Red Neuronal creado y entrenado

#### Ahora se va a proceder a verificar a cuantos datos de un data set con valores distintos a los que utilizó el modelo para entrenarse, el modelo puede predecir correctamente su clase

In [ ]:
# Creamos un diccionario para los valores de cada clase a la que puede pertenecer un sub set de datos
diccionario = {
  0: "T-shirt/top",
  1: "Trouser",
  2: "Pullover",
  3: "Dress",
  4: "Coat",
  5: "Sandal",
  6: "Shirt",
  7: "Sneaker",
  8: "Bag",
  9: "Ankle boot"
}

# Se procesa el dataset separando X de Y de Test
X = datosTest.iloc[:, 1:] / 1000.0 #Normalizacion de los datos
m, n = X.shape
y = np.asarray(datosTest.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)

# Se carga el modelo de pesos de Thetas optimizados
# with (open("model_trained_110_Neurons", "rb")) as openfile:
with (open("model_trained_130_Neurons", "rb")) as openfile:
    while True:
        try:
            thetasOptimized = pickle.load(openfile)
        except EOFError:
            break

# Se extraen los shapes de las matrices thetas
theta_shapes = np.hstack((
    NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),
    (NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)
))

# Se calcula la clase a la que pertenece cada sub set de datos para predecir contra el valor real de la clase
resultados = rn.prediction(thetasOptimized, theta_shapes, X, Y)


#### Se realiza la verificación de que la cantidad de valores en los que acerto el algoritmo y en los que no

In [ ]:
# Contadores y Arrays para almacenar las predicciones y los valores esperados
contadorAciertos = 0
contadorFallos = 0

predicciones = []
valoresEsperados = []

# Se guardan los resultados en un arrays
for row1 in resultados[0]:
    result1 = np.where(row1 == np.amax(row1))
    predicciones.append(result1[0])

for row2 in resultados[1]:
    result2 = np.where(row2 == np.amax(row2))
    valoresEsperados.append(result2[0])

# Se contabilizan los aciertos, los fallos y el porcentaje de rendimiento (# aciertos/total)
for s in range(len(X)):
    if predicciones[s].item(0) == valoresEsperados[s].item(0):
        contadorAciertos = contadorAciertos + 1
    else:
        contadorFallos = contadorFallos + 1

print("Predicciones correctas: ", contadorAciertos, " de ", contadorAciertos + contadorFallos)
print("Predicciones incorrectas: ", contadorFallos, " de ", contadorAciertos + contadorFallos)
print("Porcentaje de rendimiento: ", contadorAciertos/(contadorAciertos + contadorFallos), "% de aciertos")

## Cross Validation del modelo de Red Neuronal ya testeado

#### Se procede a realizar un Cross Validation del Modelo de Red Neuronal que ya fue testeado para determinar el rendimiento final de nuestro modelo

In [ ]:
# Se procesa el dataset separando X de Y de Cross Validation
X = datosCrossValidation.iloc[:, 1:] / 1000.0 #Normalizacion de los datos
m, n = X.shape
y = np.asarray(datosCrossValidation.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)

# Se carga el modelo de pesos de Thetas optimizados
# with (open("model_trained_110_Neurons", "rb")) as openfile:
with (open("model_trained_130_Neurons", "rb")) as openfile:
    while True:
        try:
            thetasOptimized = pickle.load(openfile)
        except EOFError:
            break

# Se extraen los shapes de las matrices thetas
theta_shapes = np.hstack((
    NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),
    (NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)
))

# Se calcula la clase a la que pertenece cada sub set de datos para predecir contra el valor real de la clase
resultados = rn.prediction(thetasOptimized, theta_shapes, X, Y)

#### Se realiza la verificación de que la cantidad de valores en los que acerto el algoritmo y en los que no

In [ ]:
# Contadores y Arrays para almacenar las predicciones y los valores esperados
contadorAciertos = 0
contadorFallos = 0

predicciones = []
valoresEsperados = []

# Se guardan los resultados en un arrays
for row1 in resultados[0]:
    result1 = np.where(row1 == np.amax(row1))
    predicciones.append(result1[0])

for row2 in resultados[1]:
    result2 = np.where(row2 == np.amax(row2))
    valoresEsperados.append(result2[0])

# Se contabilizan los aciertos, los fallos y el porcentaje de rendimiento (# aciertos/total)
for s in range(len(X)):
    if predicciones[s].item(0) == valoresEsperados[s].item(0):
        contadorAciertos = contadorAciertos + 1
    else:
        contadorFallos = contadorFallos + 1

print("Predicciones correctas: ", contadorAciertos, " de ", contadorAciertos + contadorFallos)
print("Predicciones incorrectas: ", contadorFallos, " de ", contadorAciertos + contadorFallos)
print("Porcentaje de rendimiento: ", round(100 * contadorAciertos/(contadorAciertos + contadorFallos), 2), "% de aciertos")

### Análisis de los resultados

A partir de los resultado obtenidos con la creación de la Red Neuronal se pudo realizar 2 modelos distintos para determinar si el resultado cambiaba de alguna forma. Un modelo consistia en tener 1 Capa de entrada de 784 neuronas, 1 Capa oculta de 130 neuronas y 1 Capa de salida de 10 neuronas. El segundo modelo consistia en tener 1 Capa de entrada de 784 neuronas, 1 Capa oculta de 110 neuronas y 1 Capa de salida de 10 neuronas. Como se puede observar a simple vista, el cambio realizado no fue con respecto al número de capas ocultas del modelo sino con respecto a las neuronas. Se quizo realizar uno con respecto al número de capas, pero la generación de los pesos de thetas eran random, en ningun momento se logró una optimización relevante de los pesos de thetas haciendo pruebas exaustivas. 

Sin embargo, al realizar el modelo con 1 capa oculta y variando la cantidad de neuronas, si fue posible encontrar thetas iniciales que permitieran una optimización relevante para las thetas finales del modelo. A partir de la generación de ambos modelos se obtuvieron los siguientes resultados:

    - Modelo con 1 capa oculta de 130 neuronas: 84.3% de aciertos sobre el Cross Validation Set
    - Modelo con 1 capa oculta de 110 neruonas: 85.1% de aciertos sobre el Cross Validation Set

Esto último fue el resultado del test de cada modelo con el mismo cross validation, pero si se cambia el set nuevamente, los porcentajes varian, y en algunos momentos el de 130 neuronas predice mejor que el de 110 neuronas. Esto me indica que a pesar de haber hecho un cambio en la cantidad de neuronas del modelo, este no se ve afectado significativamente en su desempeño y rendimiento. Eso quiere decir que ambos modelos tuvieron un rendimiento alto de entre un 83% a un 88%. Esto es un resultado excelente y basados en la teoría de Redes Neuronales un valor aceptable y optimo para predecir. 

### Conclusiones

La conclusión más importante de este laboratorio es que el modelo logró tener entre un 83%-88% de rendimiento acertenadole a estos porcentajes con distintos grupos de Croos Validation. Esto permite observar que mi Red Neuronal tuvo una buena optimzación de sus valores al entrenarse con un set de datos inicial. El que no haya superado el 90% de rendimiento no significa que sea un mal modelo, sino que podría ser a causa de no tener valores theta iniciales optimos para entrenar al algoritmo. Otro factor que pudiera haber ayudado a obtener un resultado distinto al de los 2 modelos utilizados, sería el de poder agregar otra capa oculta al modelo. Sin embargo, esto no fue posible debido a que tambien los valores random de los pesos thetas jugaban un papel escencial al momento de optimizar bien dichos valores. Nunca se logró obtener una optimización relevante para dicha aquitectura de la red neuronal, pero probablemente el rendimiento hubiera tenido un patrón distinto, ya sea subiendo o bajando el porcentaje de rendimiento. Por lo tanto, se obtuvo un buen modelo de Red Neuronal para predecir prendas de ropa que venian en forma de datos de pixeles que conformaban la imagen, aunque hubiera sido bueno poder generar otra arquitectura diferente a la que se utilizo de solo una capa oculta, para notar diferencia entre las arquitecturas.

### PLUS del programa

#### Con el siguiente bloque de codigo es posible representar el set de datos en una imagen hecha de pixeles de 27x27 donde se muestran las clasificaciones o predicciones correctas que hizo el algoritmo y la imagen

In [ ]:
# Se hace un plot de los aciertos para mostrar la imagen de los pixeles y la clase
# a la que el algoritmo asigno el conjunto de datos de pixeles
for s in range(len(X)):
    if predicciones[s].item(0) == valoresEsperados[s].item(0):
        X = np.asarray(X)
        data = X[s]

        data = data.reshape(28,28)

        plt.title("Prediccion: " + diccionario[predicciones[s].item(0)])
        plt.imshow(data, interpolation='nearest')
        plt.xticks(np.arange(0.0, 28.5, 1), np.arange(0.5, 28, 0.5))
        plt.yticks(np.arange(28, -0.5, -1), np.arange(0.5, 28, 0.5))

        plt.show()